In [1]:
import pandas as pd 
df_raw = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/mom.csv', index_col=0)

In [2]:
df_raw = df_raw.drop(df_raw.iloc[:,8:],axis=1)

In [3]:
df_raw

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트
1,2022-12-23,10150,10600,11200,10100,8180753,86595242100,바이오노트
2,2022-12-26,9720,10550,10700,9650,4479667,45206813740,바이오노트
3,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트
4,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트
...,...,...,...,...,...,...,...,...
2621629,2022-12-23,3765,3900,3900,3765,33477,127543190,모헨즈
2621630,2022-12-26,3720,3765,3765,3610,30218,111265525,모헨즈
2621631,2022-12-27,3755,3700,3755,3665,17856,66261900,모헨즈
2621632,2022-12-28,3630,3770,3775,3615,25279,93477675,모헨즈


In [4]:
def make_momentum_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        con1 = (df.loc[idx[0]:idx[-1],'종가'].shift(-3))
        con2 = (df.loc[idx[0]:idx[-1],'종가'].shift(-2))
        con3 = (df.loc[idx[0]:idx[-1],'종가'].shift(-1))
        df.loc[idx[0]:idx[-1], 'momentum'] = (con1<con2)&(con2<con3)&(con3<(df.loc[idx[0]:idx[-1], '종가']))
    return df
        

In [5]:
df_test = make_momentum_signal(df_raw)

In [39]:
# market_timming 병합작업
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    df = df.drop(['등락률','PER','선행PER','PBR','배당수익률'],axis=1)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [55]:
start = df_test[df_test['종목명']=='삼성전자'].head(1)['날짜'].values[0]
end = df_test[df_test['종목명']=='삼성전자'].tail(1)['날짜'].values[0]

In [67]:
df_mt = making_mt(start,end,'2001')
df_mt['날짜']=df_mt.index

#### 거래대금 필터만우선적용..

In [6]:
df_test['vol_condition'] = df_test['누적거래대금'] >=10000000000

#### 최종시그널

In [7]:
def make_buysell(df):
    df['buy'] = (df['vol_condition'])&(df['momentum'])
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        df.loc[idx[0]:idx[-1],'sell'] = df.loc[idx[0]:idx[-1],'buy'].shift(3)
    return df

In [8]:
df_signal = make_buysell(df_test)

In [11]:
df_signal.head(10)

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,momentum,vol_condition,buy,sell
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트,True,True,True,NaN
1,2022-12-23,10150,10600,11200,10100,8180753,86595242100,바이오노트,True,True,True,NaN
2,2022-12-26,9720,10550,10700,9650,4479667,45206813740,바이오노트,True,True,True,NaN
3,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트,False,True,False,True
4,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트,False,True,False,True
5,2022-12-29,8580,8890,8920,8450,2059794,17721767120,바이오노트,False,True,False,True
6,2022-11-28,7420,10200,10450,7420,3458712,27586626590,유니드비티플러스,True,True,True,NaN
7,2022-11-29,6900,7330,7350,6860,834344,5849468050,유니드비티플러스,True,False,False,NaN
8,2022-11-30,6360,6960,7010,6340,855296,5697738920,유니드비티플러스,False,False,False,NaN
9,2022-12-01,6330,6470,6530,6320,431925,2764739060,유니드비티플러스,False,False,False,True


In [111]:
df_signal = df_signal.drop('test',axis=1)

In [113]:
df_result = df_signal.dropna()

In [9]:
df_result.isnull().sum()

NameError: name 'df_result' is not defined

In [12]:
code_kospi = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kosdaq_codes.csv')

In [13]:
code = pd.concat([code_kospi,code_kosdaq])

In [14]:
code.rename(columns={'회사명':'종목명'},inplace=True)

In [39]:
code['종목코드'] = code['종목코드'].astype('str')

In [40]:
code.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2438 entries, 0 to 1611
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     2438 non-null   object
 1   종목코드    2438 non-null   object
 2   상장일     2438 non-null   object
dtypes: object(3)
memory usage: 76.2+ KB


In [17]:
df_join = df_signal.merge(code,on='종목명',how='inner')

In [51]:
df_join.rename(columns={'종목코드':'코드'},inplace=True)

In [20]:
df_join['구매수량']=3

#### 백테스트

In [21]:
df_join['sell'].unique()

array([nan, True, False], dtype=object)

In [47]:
import sys
sys.path.append('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test')
from BackTesting_System.backtesting import BackTesting, Utils

In [59]:
bt1 = BackTesting()
ut = Utils()

In [60]:
bt1.MONEY = 2000000000

In [61]:
bt1.buy_sell(df=df_join_us,buy='buy',sell='sell')

In [62]:
bt1.update_information()

In [69]:
bt1.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2018-02-01,285130,74726,3,315.0,매수
1,2018-02-01,251270,173500,3,731.0,매수
2,2018-02-01,267250,92600,3,390.0,매수
3,2018-02-01,267260,55634,3,235.0,매수
4,2018-02-01,207940,438309,3,1848.0,매수
...,...,...,...,...,...,...
16814,2020-09-11,118000,4394,3,19.0,매수
16815,2020-09-14,118000,4172,3,18.0,매도
16816,2020-11-11,121800,6930,3,29.0,매수
16817,2020-11-12,121800,6900,3,29.0,매도


In [70]:
bt1.ECONO_INFORM

,날짜,매수총합,매도총합,매매비용,실현손익
0,2018-02-01,28293477,0.0,39756.0,0.0
1,2018-02-02,27523809,810972,39818.0,-29322
2,2018-02-05,8506431,2317878,15208.0,-173049.0
3,2018-02-06,10390860,6858318,24240.0,-634065.0
4,2018-02-07,3539592,690504,5945.0,-102405
...,...,...,...,...,...
625,2020-09-11,13182,0.0,19.0,0.0
626,2020-09-14,0.0,12516,18.0,-666
627,2020-11-11,20790,0.0,29.0,0.0
628,2020-11-12,0.0,20700,29.0,-90


In [71]:
bt1.ECONO_INFORM['실현손익'].sum()

-29282604.0

In [74]:
bt1.ECONO_INFORM['실현손익'].sum()/2000000000*100

-1.4641302

In [67]:
bt1.ASSET

{'079550': {'평단가': 1.5097851686534996e+16, '수량': 33},
 '018260': {'평단가': 1.0712392003107447e+83, '수량': 138},
 '020150': {'평단가': 5.580754456651031e+73, '수량': 126},
 '034730': {'평단가': 4.170883893834454e+111, '수량': 177},
 '039490': {'평단가': 1.7312131387527363e+42, '수량': 78},
 '035420': {'평단가': 1.5431518513590183e+68, '수량': 117},
 '011070': {'평단가': 2.2198932794596897e+91, '수량': 150},
 '097950': {'평단가': 1.4201600709928806e+96, '수량': 156},
 '028670': {'평단가': 2.5032116990695676e+35, '수량': 69},
 '096770': {'평단가': 2.91552119003347e+120, '수량': 189},
 '078930': {'평단가': 1.0867915532031254e+42, '수량': 78},
 '036570': {'평단가': 2.5533698942155848e+85, '수량': 141},
 '069960': {'평단가': 1.212410756329468e+21, '수량': 42},
 '066570': {'평단가': 1.3794760756884785e+111, '수량': 177},
 '051900': {'평단가': 9.412181473444957e+148, '수량': 225},
 '047050': {'평단가': 1.21290687444618e+22, '수량': 45},
 '036460': {'평단가': 4.987697492811021e+32, '수량': 63},
 '009540': {'평단가': 1.070530206832143e+118, '수량': 186},
 '030000': {'평단가': 1.0

In [35]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544895 entries, 0 to 2544894
Data columns (total 15 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   날짜             object
 1   종가             int64 
 2   시가             int64 
 3   최고가            int64 
 4   최저가            int64 
 5   누적거래량          int64 
 6   누적거래대금         int64 
 7   종목명            object
 8   momentum       object
 9   vol_condition  bool  
 10  buy            bool  
 11  sell           object
 12  코드             int64 
 13  상장일            object
 14  구매수량           int64 
dtypes: bool(2), int64(8), object(5)
memory usage: 276.7+ MB


In [68]:
df_join_us.to_csv('reuslt.csv')

In [44]:
df_join_us = df_join.merge(code,on='종목명',how='inner')

In [46]:
df_join_us['종목코드'] = df_join_us['']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544895 entries, 0 to 2544894
Data columns (total 16 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   날짜             object
 1   종가             int64 
 2   시가             int64 
 3   최고가            int64 
 4   최저가            int64 
 5   누적거래량          int64 
 6   누적거래대금         int64 
 7   종목명            object
 8   momentum       object
 9   vol_condition  bool  
 10  buy            bool  
 11  sell           object
 12  상장일_x          object
 13  구매수량           int64 
 14  종목코드           object
 15  상장일_y          object
dtypes: bool(2), int64(7), object(7)
memory usage: 296.1+ MB


In [58]:
df_join_us.rename(columns={'종목코드':'코드'},inplace=True)